In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras


print(tf.test.is_gpu_available())
print(tf.__version__)
print(sys.version_info)

for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

True
2.0.0
sys.version_info(major=3, minor=7, micro=7, releaselevel='final', serial=0)
matplotlib 3.2.2
numpy 1.18.5
pandas 1.0.5
sklearn 0.21.2
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [3]:
from sklearn.model_selection import train_test_split

x_train_all,x_test,y_train_all,y_test = train_test_split(housing.data,housing.target,random_state=7)

x_train,x_valid,y_train,y_vaild=train_test_split(x_train_all,y_train_all,random_state=11)

print(x_train.shape,y_train.shape)
print(x_valid.shape,y_vaild.shape)
print(x_test.shape,y_test.shape)

(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [4]:
# 数据标准化

from sklearn.preprocessing import StandardScaler

saclar = StandardScaler()
x_train_sacled = saclar.fit_transform(x_train)
x_valid_sacled = saclar.transform(x_valid)
x_test_sacled=saclar.transform(x_test)

In [5]:
# metric使用
# 均方差
metric = keras.metrics.MeanSquaredError()
print(metric([5.],[2]))
print(metric([0.],[1.]))
# 可以累加
print(metric.result())

# 不累加
metric.reset_states()

print(metric([1.],[3]))

tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [6]:
# tf.GradientTape手动求导方式进行回归问题的求解
epochs= 100
batch_size = 32
steps_per_epoch = len(x_train_sacled) // batch_size
optimizer = keras.optimizers.SGD()
metric = keras.metrics.MeanSquaredError()

def random_batch(x,y,batch_size=32):
    idx = np.random.randint(0,len(x),size=batch_size)
    return x[idx],y[idx]




model = keras.models.Sequential([
 
    keras.layers.Dense(30,activation="relu",input_shape=x_train.shape[1:]),
    keras.layers.Dense(1),
])


for epoch in range(epochs):
    metric.reset_states()
    for step in range(steps_per_epoch):
        x_batch,y_batch = random_batch(x_test_sacled,y_train,batch_size)
        
        with tf.GradientTape() as tape:
            y_pred = model(x_batch)
            loss = tf.reduce_mean(keras.losses.mean_squared_error(y_batch,y_pred))
            metric(y_batch,y_pred)
        grads = tape.gradient(loss,model.variables)
        grads_and_vars = zip(grads,model.variables)
        optimizer.apply_gradients(grads_and_vars)
        print("\rEpoch",epoch,"train mse:",metric.result().numpy(),end="")
    y_vaild_pred = model(x_valid_sacled)
    valid_loss = tf.reduce_mean(
                    keras.losses.mean_squared_error(y_vaild_pred,y_vaild))
    print("\t","valid mse:",valid_loss.numpy())
# model.summary()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 0 train mse: 1.7637064	 valid mse: 5.072939349954521 mse: 2.4532516 0 train mse: 2.1564357 0 train mse: 1.9521608 0 train mse: 1.8144933 0 train mse: 1.7618549 0 train mse: 1.7295204 0 train mse: 1.7041945 0 train mse: 1.6288625 0 train mse: 1.8373284 0 train mse: 1.8038394 0 train mse: 1.7903012
Epoch 1 train mse: nan	 valid mse: nanmse: 1.3456103 1 train mse: 1.4032915 1 train mse: 1.4431543 1 train mse: 1.4122431 1 train mse: 1.382667 1 train mse: 1.394864 1 train mse: 1.4556907 1 train mse: 1.

Epoch 31 train mse: nan	 valid mse: nanan 31 train mse: nan 31 train mse: nan 31 train mse: nan 31 train mse: nan 31 train mse: nan 31 train mse: nan 31 train mse: nan 31 train mse: nannan 31 train mse: nan 31 train mse: nan
Epoch 32 train mse: nan 32 train mse: nan 32 train mse: nan 32 train mse: nan 32 train mse: nan 32 train mse: nan 32 train mse: nan 32 train mse: nan 32 train mse: nan 32 train mse: nan 32 train mse: nan 32 train mse: nannan 32 train mse: nan 32 train mse: nan	 valid mse: nan
Epoch 33 train mse: nan	 valid mse: nanan 33 train mse: nan 33 train mse: nan 33 train mse: nan 33 train mse: nan 33 train mse: nan 33 train mse: nan train mse: nan 33 train mse: nan 33 train mse: nan 33 train mse: nan 33 train mse: nan 33 train mse: nan
Epoch 34 train mse: nan	 valid mse: nanan 34 train mse: nan 34 train mse: nan 34 train mse: nan 34 train mse: nan 34 train mse: nan nan 34 train mse: nan 34 train mse: nan 34 train mse: nan 34 train mse: nan 34 train mse: nan
Epoch 35 train ms

Epoch 65 train mse: nan 65 train mse: nan 65 train mse: nan65 train mse: nan 65 train mse: nan 65 train mse: nan 65 train mse: nan 65 train mse: nan 65 train mse: nan 65 train mse: nan 65 train mse: nan 65 train mse: nan 65 train mse: nan	 valid mse: nan
Epoch 66 train mse: nan	 valid mse: nanan 66 train mse: nan 66 train mse: nan 66 train mse: nan 66 train mse: nan 66 train mse: nan 66 train mse: nan 66 train mse: nan 66 train mse: nan 66 train mse: nan 66 train mse: nan 66 train mse: nan
Epoch 67 train mse: nan 67 train mse: nan 67 train mse: nan 67 train mse: nan 67 train mse: nan 67 train mse: nan 67 train mse: nan 67 train mse: nan 67 train mse: nan 67 train mse: nan 67 train mse: nan 67 train mse: nan	 valid mse: nan
Epoch 68 train mse: nan	 valid mse: nane: nan 68 train mse: nan 68 train mse: nan 68 train mse: nan 68 train mse: nan68 train mse: nan 68 train mse: nan 68 train mse: nan 68 train mse: nan 68 train mse: nan
Epoch 69 train mse: nan	 valid mse: nanan 69 train mse: nan 